In [3]:
import numpy as np
from numba import jit,vectorize,float64, int64
import pdb

In [21]:
def T_mat_template(kbar):
    # Kbar2 = 2^3 = 8
    kbar2 = 2**kbar
    A = np.zeros((kbar2,kbar2))
    for i in range(kbar2):
        for j in range(i,kbar2-i):
            A[i,j] = np.bitwise_xor(i,j)
    return(A)     

In [12]:
@jit(float64[:,:](float64[:,:],float64[:],int64))
def transition_mat_jit(A,inpt,kbar):
    b = inpt[0]
    gamma_kbar = inpt[2]
    gamma = np.zeros((kbar,1))
    gamma[0,0] = 1-(1-gamma_kbar)**(1/(b**(kbar-1)))
    for i in range(1,kbar):
        gamma[i,0] = 1-(1-gamma[0,0])**(b**(i))
    gamma = gamma*0.5
    gamma = np.c_[gamma,gamma]
    gamma[:,0] = 1 - gamma[:,1]
    kbar2 = 2**kbar
    prob = np.ones((kbar2,1))
    
    for i in range(kbar2):
        for m in range(kbar):
            prob[i,0] =prob[i,0] * gamma[kbar-m-1,
                np.unpackbits(np.array([i],dtype = np.uint8))[-(m+1)]]
    for i in range(2**(kbar-1)):
        for j in range(i,2**(kbar-1)):
            A[kbar2-i-1,j] = prob[np.rint(kbar2 - A[i,j]-1).astype(int),0]
            A[kbar2-j-1,i] = A[kbar2-i-1,j]
            A[j,kbar2-i-1] = A[kbar2-i-1,j]
            A[i,kbar2-j-1] = A[kbar2-i-1,j]
            A[i,j] = prob[np.rint(A[i,j]).astype(int),0]
            A[j,i] = A[i,j].copy()
            A[kbar2-j-1,kbar2-i-1] = A[i,j]
            A[kbar2-i-1,kbar2-j-1] = A[i,j]
        
    return(A)

In [31]:
def transition_mat(A,inpt,kbar):
    b = inpt[0]
    gamma_kbar = inpt[2]
    gamma = np.zeros((kbar,1))
    gamma[0,0] = 1-(1-gamma_kbar)**(1/(b**(kbar-1)))
    for i in range(1,kbar):
        gamma[i,0] = 1-(1-gamma[0,0])**(b**(i))
    gamma = gamma*0.5
    gamma = np.c_[gamma,gamma]
    gamma[:,0] = 1 - gamma[:,1]
    kbar2 = 2**kbar
    prob = np.ones((kbar2,1))
    
    for i in range(kbar2):
        for m in range(kbar):
            prob[i,0] =prob[i,0] * gamma[kbar-m-1,
                np.unpackbits(np.array([i],dtype = np.uint8))[-(m+1)]]
            
    for i in range(2**(kbar-1)):
        for j in range(i,2**(kbar-1)):
            A[kbar2-i-1,j] = prob[np.rint(kbar2 - A[i,j]-1).astype(int),0]
            A[kbar2-j-1,i] = A[kbar2-i-1,j]
            A[j,kbar2-i-1] = A[kbar2-i-1,j]
            A[i,kbar2-j-1] = A[kbar2-i-1,j]
            A[i,j] = prob[np.rint(A[i,j]).astype(int),0]
            A[j,i] = A[i,j].copy()
            A[kbar2-j-1,kbar2-i-1] = A[i,j]
            A[kbar2-i-1,kbar2-j-1] = A[i,j]
        
    return(A)

In [22]:
A = [T_mat_template(i) for i in range(2,9)]

In [75]:
%%timeit
transition_mat_jit(A[0],np.array([3,1.5,0.5,3]),2)

163 µs ± 6.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [76]:
%%timeit
transition_mat(A[0],np.array([3,1.5,0.5,3]),2)

80.5 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [77]:
%%timeit
transition_mat_jit(A[1],np.array([3,1.5,0.5,3]),3)

244 µs ± 5.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [78]:
%%timeit
transition_mat(A[1],[3,1.5,0.5,3],3)

171 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [79]:
%%timeit
transition_mat_jit(A[2],np.array([3,1.5,0.5,3]),4)

537 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [80]:
%%timeit
transition_mat(A[2],np.array([3,1.5,0.5,3]),4)

455 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [81]:
%%timeit
transition_mat_jit(A[3],np.array([3,1.5,0.5,3]),5)

1.47 ms ± 24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [82]:
%%timeit
transition_mat(A[3],np.array([3,1.5,0.5,3]),5)

1.4 ms ± 58.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [83]:
%%timeit
transition_mat_jit(A[4],np.array([3,1.5,0.5,3]),6)

4.78 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
%%timeit
transition_mat(A[4],np.array([3,1.5,0.5,3]),6)

4.79 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
%%timeit
transition_mat_jit(A[5],np.array([3,1.5,0.5,3]),7)

16.9 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [86]:
%%timeit
transition_mat(A[5],np.array([3,1.5,0.5,3]),7)

17.3 ms ± 361 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [87]:
%%timeit
transition_mat_jit(A[6],np.array([3,1.5,0.5,3]),8)

62.8 ms ± 825 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [88]:
%%timeit
transition_mat(A[6],np.array([3,1.5,0.5,3]),8)

63.7 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
def MSM_likelihood(inpt,kbar,data,A_template,estim_flag,nargout =1):
    if not hasattr(inpt,"__len__"):
        inpt = [estim_flag[0],inpt,estim_flag[1],estim_flag[2]]
        
    sigma = inpt[3]/np.sqrt(252)
    k2 = 2**kbar
    A = transition_mat(A_template.copy(),inpt,kbar)
    g_m = gofm(inpt,kbar)
    T = len(data)
    pi_mat = np.zeros((T+1,k2))
    LLs = np.zeros(T)
    pi_mat[0,:] = (1/k2)*np.ones((1,k2))
    """
    Likelihood Algorithm
    """
    pa = (2*np.pi)**(-0.5)
    s = sigma*g_m
    w_t = data 
    w_t = pa*np.exp(-0.5*((w_t/s)**2))/s
    w_t = w_t + 1e-16

    for t in range(T):
        
        piA = np.dot(pi_mat[t,:],A)
        C = (w_t[t,:]*piA)
        ft = np.sum(C) # log
        if np.isclose(ft,0):
            pi_mat[t+1,1]=1
        else:
            pi_mat[t+1,:] = C/ft
        LLs[t] = np.log(np.dot(w_t[t,:],piA))
        
    LL = -np.sum(LLs)
    if np.any(np.isinf(LLs)):
        print("Log-likelihood is inf. Probably due to all zeros in pi_mat.")
    if nargout == 1:
        return(LL)
    else:
        return(LL,LLs)